In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://de-zoomcamp.europe-west2-c.c.cedar-style-412618.internal:7077") \
    .appName("cluster_spark") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/07 17:16:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
df_green = spark.read.parquet("data/pq/green/*/*")

In [4]:
df_yellow = spark.read.parquet("data/pq/yellow/*/*")

In [5]:
df_green = df_green \
  .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime") \
  .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

df_yellow = df_yellow \
  .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
  .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")

In [6]:
common_columns = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [7]:
from pyspark.sql import functions as F

In [8]:
df_green_sel = df_green \
  .select(common_columns)  \
  .withColumn("service_type", F.lit("green"))

df_yellow_sel = df_yellow \
  .select(common_columns)  \
  .withColumn("service_type", F.lit("yellow"))

In [9]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [10]:
df_trips_data.groupBy("service_type").count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+

24/03/07 17:56:17 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/03/07 17:56:17 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbo